In [4]:
import pandas as pd
data=pd.read_csv("FAOLEX_ALL.csv")
print(data.keys())

Index(['Record Id', 'Record URL', 'Document URL', 'Title', 'Original title',
       'Date of original text', 'Last amended date', 'Available website',
       'Language of document', 'Country/Territory', 'Regional organizations',
       'Territorial subdivision', 'Type of text', 'Repealed', 'Abstract',
       'Primary subjects', 'Domain', 'Keywords'],
      dtype='object')


In [5]:
import nltk
keywords=list(data["Keywords"])
country=list(data["Country/Territory"])
i=0
# Split alle keywords op i en liste og ændrer nan til at være en str med Nan
for words in keywords:
    if type(words)!=str:
        #print("type= ",type(words)," Index = ", i)
        words="Nan"
    if type(country[i])!=str:
        country[i]="NaN"
    keywords[i]=words.split(";")
    i+=1



In [6]:
# prints keywords for a single text
print(keywords[0])

['maritime zone', ' territorial sea', ' sovereignty']


In [7]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import numpy as np
stemmer = SnowballStemmer("english")
stemmed_words=[]

# Stem keywords
for i in range(len(keywords)):
    stemmed_words.append([stemmer.stem(word) for word in keywords[i]])



In [8]:
print(keywords[14083])
print(stemmed_words[14083])

['marine fisheries', ' molluscs', ' fishing authorization', ' non-governmental entity', ' fishery management and conservation']
['marine fisheri', ' mollusc', ' fishing author', ' non-governmental ent', ' fishery management and conserv']


In [9]:
# Count number documents in a language
languages=list(data["Language of document"])
print(languages.count("English"))

54324


In [10]:
# Make a top2vec topic model
from top2vec import Top2Vec
i=0
for word in keywords:
    keywords[i]="".join(word)
    i+=1
model = Top2Vec(keywords,embedding_model="doc2vec")


2023-05-29 16:11:01,652 - top2vec - INFO - Pre-processing documents for training
2023-05-29 16:11:09,871 - top2vec - INFO - Creating joint document/word embedding
2023-05-29 16:15:44,359 - top2vec - INFO - Creating lower dimension embedding of documents
2023-05-29 16:17:30,608 - top2vec - INFO - Finding dense areas of documents
2023-05-29 16:17:38,471 - top2vec - INFO - Finding topics


In [11]:
topic_sizes, topic_nums = model.get_topic_sizes()
topic_sizes

array([1410, 1365, 1300, ...,   22,   21,   20], dtype=int64)

In [12]:
topic_words,word_scores,topic_nums = model.get_topics(10)

# Topic words: Ord i en given topic
#Words_score: Hvor 

In [13]:
print(topic_words)

[['continental' 'territorial' 'shelf' 'maritime' 'eez' 'sea' 'exclusive'
  'economic' 'zone' 'seas' 'high' 'deep' 'sovereignty' 'transshipment'
  'fishing' 'catch' 'port' 'navigation' 'bycatch' 'fisheries' 'coastal'
  'mesh' 'allowable' 'diadromous' 'vessel' 'total' 'foreign' 'marine'
  'bed' 'method' 'cartilaginous' 'harbour' 'fishery' 'landing' 'charge'
  'migratory' 'mammals' 'exploration' 'fishes' 'state' 'iuu' 'size'
  'illegal' 'stock' 'unreported' 'unregulated' 'seasons' 'commercial'
  'turtles' 'gear']
 ['mesh' 'fishing' 'fishes' 'seasons' 'method' 'fisheries' 'allowable'
  'bycatch' 'catch' 'size' 'cartilaginous' 'total' 'charge' 'vessel'
  'port' 'fishery' 'landing' 'crustaceans' 'gear' 'diadromous'
  'commercial' 'quota' 'transshipment' 'artisanal' 'prohibited' 'turtles'
  'foreign' 'allocation' 'molluscs' 'seas' 'high' 'migratory' 'marine'
  'mammals' 'stock' 'territorial' 'exclusive' 'unregulated' 'unreported'
  'iuu' 'illegal' 'eez' 'state' 'repopulation' 'sea' 'enhanceme

In [14]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=2, num_docs=5)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Document: 19012, Score: 0.9775562286376953
-----------
timber extraction/logging authorization/permit forest management/forest conservation
-----------

Document: 112243, Score: 0.974977970123291
-----------
forest management/forest conservation timber timber extraction/logging
-----------

Document: 30936, Score: 0.9747377038002014
-----------
forest management/forest conservation timber extraction/logging
-----------

Document: 57084, Score: 0.9746578931808472
-----------
forest management/forest conservation timber timber extraction/logging
-----------

Document: 1750, Score: 0.9742560386657715
-----------
timber extraction/logging protected area forest management/forest conservation
-----------



In [ ]:

topic_words,word_scores,topic_nums = model.get_topics()
tal = max(topic_nums)
#rækker = topic, colums = ord
co_occurrence_matrix = np.zeros((tal,tal))
for i in range(tal):
    topic_words, w_score, topic_scores, t_nums = model.search_topics_by_vector(model.topic_vectors[i],tal)
    for dist, topic_nb in zip(topic_scores, t_nums):
        co_occurrence_matrix[i-1,topic_nb-1] = dist

print(co_occurrence_matrix)



